In [2]:
from tensorflow.keras import layers, Model, preprocessing
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import numpy as np

from utils import make_predictions, evaluation, data_loader

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [4]:
train_ds, val_ds, test_ds, class_names, image_size = data_loader.load_data(RANDOM_SEED, True, 0.75)

Found 15780 files belonging to 11 classes.
Using 11046 files for training.
Found 15780 files belonging to 11 classes.
Using 4734 files for validation.

Classes: ['tomato Fusarium Wilt', 'tomato spider mites', 'tomato verticillium wilt', 'tomato_bacterial_spot', 'tomato_early_blight', 'tomato_healthy_leaf', 'tomato_late_blight', 'tomato_leaf_curl', 'tomato_leaf_miner', 'tomato_leaf_mold', 'tomato_septoria_leaf']

✅ Balanced minority-only augmentation is ON

Counting class frequencies...
tomato Fusarium Wilt: 292
tomato spider mites: 456
tomato verticillium wilt: 376
tomato_bacterial_spot: 1201
tomato_early_blight: 1368
tomato_healthy_leaf: 1311
tomato_late_blight: 982
tomato_leaf_curl: 1335
tomato_leaf_miner: 1289
tomato_leaf_mold: 1342
tomato_septoria_leaf: 1094

Minority class: tomato Fusarium Wilt (count=292) → repeating 4× with augmentation

Minority class: tomato spider mites (count=456) → repeating 3× with augmentation

Minority class: tomato verticillium wilt (count=376) → repeat

In [5]:
# efficientnet base model

efficientNetBaseArchitecture = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3),
    pooling='avg'
)

# Freezing the early layers (retain general features)
efficientNetBaseArchitecture.trainable = True
for layer in efficientNetBaseArchitecture.layers:
    # All layers before 'block7a_expand_conv' are frozen
    if layer.name == 'block7a_expand_conv':
        break
    layer.trainable = False

# Custom classification head
x = efficientNetBaseArchitecture.output
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(len(class_names), activation='softmax')(x)

efficientNet = Model(inputs=efficientNetBaseArchitecture.input, outputs=predictions)

efficientNet.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

efficientNet.summary()

16705208/16705208 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)     

In [ ]:
history = efficientNet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
evaluation.plot_loss(history)
evaluation.plot_accuracy(history)

In [ ]:
y_true, y_pred = make_predictions.predict_test_data(efficientNet, test_ds)

In [ ]:
evaluation.make_classification_report(y_true, y_pred, class_names)

In [ ]:
evaluation.make_confusion_matrix(y_true, y_pred, class_names)